## <font color=darkblue> Barrier option pricing (Up-and-out) </font>

### Up-and-Out (UAO) European Call Option

S = 100 <BR>
L = 130<BR>
K = 100<BR>

In [18]:
import numpy as np
import pandas as pd

In [19]:
S = 100
L = 130
K = 100

u = 1.30
d = 1/u

In [20]:
def recursive_name(u,d,n):
    if n <=1:
        return [u,d]
    else:
        te = recursive_name(u,d,n-1)
        total_list = []
        for i2 in te:
            for i1 in [u,d]:
                total_list.append(i2+i1)
        return total_list

In [21]:
def cal_exotic_option(s,x,l,u,d,n,t,r):
    p = (1-d)/(u-d)
    p_inv = 1-p
    temp = np.ones((2**n,n+2))*s
    price_df = pd.DataFrame(temp, columns=['w']+[f'P({i})' for i in range(n+1)])
    price_df['w'] = recursive_name('u','d',n)
    def cal_price_prob(a1,i1,u1,d1,s1,t1):
        te_str = a1['w'][:i1]
        u_count = te_str.count('u')
        d_count = te_str.count('d')
        if t1=='price':
            return s1*(u1**u_count)*(d1**d_count)
        elif t1=='prob':
            return (p**u_count)*(p_inv**d_count)
    for i in range(n+1):
        price_df[f'P({i})'] = price_df.apply(cal_price_prob,args=[i,u,d,s,'price'],axis=1)
    price_df['Prob'] = price_df.apply(cal_price_prob,args=[i,u,d,s,'prob'],axis=1)
    price_df['Value']= s
    price_df_temp = price_df[[f'P({i})' for i in range(n+1)]]
    for i in range(len(price_df.index)):
        ma = max(price_df_temp.loc[i].values)
        if ma > l:
            price_df['Value'].loc[i] = 0
        else:
            if price_df[f'P({n})'].loc[i] <= x:
                price_df['Value'].loc[i] = 0
            else:
                price_df['Value'].loc[i] = price_df[f'P({n})'].loc[i] - x
                
    price_df['Temp'] =price_df['Prob'] *price_df['Value']
    print(f'The value of UAO European call option is: {sum(price_df["Temp"].values):.3f}')
    price_df = price_df.drop(['Temp'],axis=1)
    return price_df

In [22]:
cal_exotic_option(S,K,L,u,d,5,'UAO',0).head(15)

The value of UAO European call option is: 3.939


C:\Users\ragha\anaconda3\envs\WQU\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,w,P(0),P(1),P(2),P(3),P(4),P(5),Prob,Value
0,uuuuu,100.0,130.0,169.0,219.700000,285.610000,371.293000,0.015537,0.0
1,uuuud,100.0,130.0,169.0,219.700000,285.610000,219.700000,0.020198,0.0
2,uuudu,100.0,130.0,169.0,219.700000,169.000000,219.700000,0.020198,0.0
3,uuudd,100.0,130.0,169.0,219.700000,169.000000,130.000000,0.026257,0.0
4,uuduu,100.0,130.0,169.0,130.000000,169.000000,219.700000,0.020198,0.0
5,uudud,100.0,130.0,169.0,130.000000,169.000000,130.000000,0.026257,0.0
6,uuddu,100.0,130.0,169.0,130.000000,100.000000,130.000000,0.026257,0.0
7,uuddd,100.0,130.0,169.0,130.000000,100.000000,76.923077,0.034134,0.0
8,uduuu,100.0,130.0,100.0,130.000000,169.000000,219.700000,0.020198,0.0
9,uduud,100.0,130.0,100.0,130.000000,169.000000,130.000000,0.026257,0.0


The value of UAO European call option is: 3.939

### Same parameters, European call
The European call price as below calculation is USD 24.18. Compared with the UAO European call of USD 3.939 (part 3a), the European call price is more expensive than the UAO European call

In [24]:
call, put, european, american = 100, 101, 102, 103

In [25]:
side = call  # Option side
style = european  # Option style
price = 100  #Current instrument price
strike = 100  #Strike price
riskfree = 0  
divyield = 0
tte = 365  # Time to expiration in days
n = 5  # Depth of binomial tree (levels are numbered from 0 to n)
tdelta = tte / (n * 365)  # Time delta per one step (as fraction of year)
u = 1.30  # Up movement per step
d = 1 / u  # Down movement per step
rf = 0  # Risk-free rate per step
dy = 0  # Dividend yield per step
pu = (1 + rf - dy - d) / (u - d)  # Probability of up movement
pd = 1 - pu  # Probability of down movement

assert side in [call, put] and style in [american, european]
print('%18s : %0.8f' % ('Node prob U', pu))
print('%18s : %0.8f' % ('Node prob D', pd))
print('%18s : %0.8f' % ('Node tdelta', tdelta))
print('%18s : %0.8f' % ('Node discount f', rf))

       Node prob U : 0.43478261
       Node prob D : 0.56521739
       Node tdelta : 0.20000000
   Node discount f : 0.00000000


In [35]:
# Generate terminal nodes of binomial tree
level = []
print()
for j in range(0, n + 1):
    pr = price * d ** j * u ** (n - j)
    # Option value at the node (depending on side)
    ov = max(0.0, pr - strike) if side == call else max(0.0, strike - pr)
    level.append((pr, ov))
    print('Node [%i,%i] \t Price %6.3f \t Option Value %6.3f' % (n, j, pr, ov))

levels = [None, None, None]

# reduce binomial tree
for i in range(n - 1, -1, -1):  # [n-1 to 0]
    levelNext = []
    print()
    for j in range(0, i + 1):  
        node_u, node_d = level[j], level[j + 1]
        # Instrument's price at the node
        pr = node_d[0] / d
        # Option value at the node (depending on side)
        ov = (node_d[1] * pd + node_u[1] * pu) / (1 + rf)
        if style == american:  # for American options - not relevant for this assignment
            ov = max(ov, pr - strike if side == call else strike - pr)
        levelNext.append((pr, ov))
        print('Node [%i,%i] \t Price %6.3f \t Option Value %6.3f' % (i, j, pr, ov))
    level = levelNext
    if j <= 2: levels[j] = level 


Node [5,0] 	 Price 371.293 	 Option Value 271.293
Node [5,1] 	 Price 219.700 	 Option Value 119.700
Node [5,2] 	 Price 130.000 	 Option Value 30.000
Node [5,3] 	 Price 76.923 	 Option Value  0.000
Node [5,4] 	 Price 45.517 	 Option Value  0.000
Node [5,5] 	 Price 26.933 	 Option Value  0.000

Node [4,0] 	 Price 285.610 	 Option Value 185.610
Node [4,1] 	 Price 169.000 	 Option Value 69.000
Node [4,2] 	 Price 100.000 	 Option Value 13.043
Node [4,3] 	 Price 59.172 	 Option Value  0.000
Node [4,4] 	 Price 35.013 	 Option Value  0.000

Node [3,0] 	 Price 219.700 	 Option Value 119.700
Node [3,1] 	 Price 130.000 	 Option Value 37.372
Node [3,2] 	 Price 76.923 	 Option Value  5.671
Node [3,3] 	 Price 45.517 	 Option Value  0.000

Node [2,0] 	 Price 169.000 	 Option Value 73.167
Node [2,1] 	 Price 100.000 	 Option Value 19.454
Node [2,2] 	 Price 59.172 	 Option Value  2.466

Node [1,0] 	 Price 130.000 	 Option Value 42.808
Node [1,1] 	 Price 76.923 	 Option Value  9.852

Node [0,0] 	 Price 

### UAO call vs Vanilla call option

* UAO European call options are cheaper than Vanilla options. Thus, it requires lower cash outlay
* UAO options are an OTC product and can be customised 
* They allow traders and institutions to hedge one-sided risk efficiently 


### Price of UAI 

Up and In option allows the holders the right to buy or sell underlying asset when the price of the underlying asset matches or exceeds a barrier level. It is opposite to the up and out option. The price of a up and in option could be determined by the European call minus the up and out price.
<br><br>
In our case, 
* UAO price = 3.39
* Vanilla price = 24.18
<br><br>
Therefore, the price of the UAI call will be 24.18-3.93 = 20.3